In [ ]:
%pip install langchain-openai langchain-core langchain-community 

In [29]:
system_instructions = """
You are a converter that transforms REST API endpoint specs (OpenAPI or Postman-like endpoint descriptions) into a W3C Thing Description (TD) JSON. Output ONLY valid JSON (no explanation). Follow the mapping rules:
- GET that returns resource -> property (readOnly if no write endpoint).
- POST/PUT/PATCH/DELETE -> action.
- Webhooks/SSE -> events.
Include: "@context", "id", "title", "security", "properties", "actions", "events",
and for each affordance include at least one "form" with {{ "href", "op", "contentType" }}.
Make names short. For path params: /booking/{{id}}.
"""

In [30]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY") or getpass.getpass("Enter OpenAI API Key: ")

In [31]:
from langchain_openai import ChatOpenAI

openai_model = "gpt-4.1"
# For normal accurate responses
llm = ChatOpenAI(temperature=0.0, model=openai_model)

In [32]:
user_prompt = """Convert the following endpoint to TD JSON only
    INPUT: \n {endpoint_spec}
    """

In [33]:
from langchain_core.prompts import ChatPromptTemplate

example_prompt = ChatPromptTemplate.from_messages([
    ("human", user_prompt),
    ("ai", "{generated_td}"),
])

In [34]:
example1 = {
    "endpoint_spec": """Endpoint: GET /books/{id}
                    Description: Returns a book by its ID. Response: { "id": "123", "title": "X", "author": "Y" }
                    AUTH: Bearer token""",
    "generated_td": """{
                    "@context": "https://www.w3.org/2019/wot/td/v1",
                    "id": "urn:example:jobService",
                    "title": "JobService",
                    "security": ["api_key"],
                    "securityDefinitions": { "api_key": { "scheme": "apikey", "in": "header", "name": "x-api-key" } },
                    "actions": {
                        "createJob": {
                        "title": "createJob",
                        "input": {
                            "type": "object",
                            "properties": {
                            "profile": { "type": "string" },
                            "priority": { "type": "string", "enum": ["low", "high"] }
                            },
                            "required": ["profile"]
                        },
                        "forms": [
                            { "href": "https://api.example.com/jobs", "op": ["invokeaction"], "contentType": "application/json" }
                        ]
                        }
                    }
                    }""",
}


In [35]:
example2 = {
    "endpoint_spec": """Endpoint: POST /jobs
                    Description: Create a new job. Body: { "profile": "...", "priority": "low|high" }. Returns 201 with job id.
                    AUTH: API Key in header "x-api-key""",
    "generated_td": """{
                        "@context": "https://www.w3.org/2019/wot/td/v1",
                        "id": "urn:example:jobService",
                        "title": "JobService",
                        "security": ["api_key"],
                        "securityDefinitions": { "api_key": { "scheme": "apikey", "in": "header", "name": "x-api-key" } },
                        "actions": {
                            "createJob": {
                            "title": "createJob",
                            "input": {
                                "type": "object",
                                "properties": {
                                "profile": { "type": "string" },
                                "priority": { "type": "string", "enum": ["low", "high"] }
                                },
                                "required": ["profile"]
                            },
                            "forms": [
                                { "href": "https://api.example.com/jobs", "op": ["invokeaction"], "contentType": "application/json" }
                            ]
                            }
                        }
                        }""",
}


In [36]:
examples = [example1, example2]

In [37]:
from langchain_core.prompts import FewShotChatMessagePromptTemplate

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples
)

print(few_shot_prompt.format())

Human: Convert the following endpoint to TD JSON only
    INPUT: 
 Endpoint: GET /books/{id}
                    Description: Returns a book by its ID. Response: { "id": "123", "title": "X", "author": "Y" }
                    AUTH: Bearer token
    
AI: {
                    "@context": "https://www.w3.org/2019/wot/td/v1",
                    "id": "urn:example:jobService",
                    "title": "JobService",
                    "security": ["api_key"],
                    "securityDefinitions": { "api_key": { "scheme": "apikey", "in": "header", "name": "x-api-key" } },
                    "actions": {
                        "createJob": {
                        "title": "createJob",
                        "input": {
                            "type": "object",
                            "properties": {
                            "profile": { "type": "string" },
                            "priority": { "type": "string", "enum": ["low", "high"] }
                         

In [38]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", system_instructions),
        few_shot_prompt,
        ("user", user_prompt),
    ]
)

In [39]:
prompt_template.input_variables

['endpoint_spec']

In [44]:
print(prompt_template.format)

<bound method BaseChatPromptTemplate.format of ChatPromptTemplate(input_variables=['endpoint_spec'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='\nYou are a converter that transforms REST API endpoint specs (OpenAPI or Postman-like endpoint descriptions) into a W3C Thing Description (TD) JSON. Output ONLY valid JSON (no explanation). Follow the mapping rules:\n- GET that returns resource -> property (readOnly if no write endpoint).\n- POST/PUT/PATCH/DELETE -> action.\n- Webhooks/SSE -> events.\nInclude: "@context", "id", "title", "security", "properties", "actions", "events",\nand for each affordance include at least one "form" with {{ "href", "op", "contentType" }}.\nMake names short. For path params: /booking/{{id}}.\n'), additional_kwargs={}), FewShotChatMessagePromptTemplate(examples=[{'endpoint_spec': 'Endpoint: GET /books/{id}\n                    Description:

In [51]:
pipeline = (
    {"endpoint_spec": lambda x: x["endpoint_spec"]}
    | prompt_template
    | llm
    | {"generated_td": lambda x: x.content}
)

In [52]:
get_booking_id = """
Endpoint: GET https://restful-booker.herokuapp.com/booking/
Description: 

Returns the ids of all the bookings that exist within the API.  

Response: HTTP/1.1 200 OK
[
  {
    "bookingid": 1
  },
  {
    "bookingid": 2
  },
  {
    "bookingid": 3
  },
  {
    "bookingid": 4
  }
]

AUTH: Bearer token
"""

In [53]:
response = pipeline.invoke({
    "endpoint_spec": get_booking_id
})

In [54]:
response

{'generated_td': '{\n  "@context": "https://www.w3.org/2019/wot/td/v1",\n  "id": "urn:example:bookingService",\n  "title": "BookingService",\n  "security": ["bearer_token"],\n  "securityDefinitions": {\n    "bearer_token": {\n      "scheme": "bearer"\n    }\n  },\n  "properties": {\n    "bookings": {\n      "title": "bookings",\n      "description": "Returns the ids of all the bookings that exist within the API.",\n      "type": "array",\n      "items": {\n        "type": "object",\n        "properties": {\n          "bookingid": { "type": "integer" }\n        },\n        "required": ["bookingid"]\n      },\n      "readOnly": true,\n      "forms": [\n        {\n          "href": "https://restful-booker.herokuapp.com/booking/",\n          "op": ["readproperty"],\n          "contentType": "application/json"\n        }\n      ]\n    }\n  },\n  "actions": {},\n  "events": {}\n}'}

In [ ]:
response

{'generated_workflow': '{\n  "@context": "https://www.w3.org/2019/wot/td/v1",\n  "id": "urn:example:bookingService",\n  "title": "BookingService",\n  "security": ["bearer_token"],\n  "securityDefinitions": {\n    "bearer_token": {\n      "scheme": "bearer"\n    }\n  },\n  "properties": {\n    "bookings": {\n      "title": "Bookings",\n      "description": "Returns the ids of all the bookings that exist within the API.",\n      "type": "array",\n      "items": {\n        "type": "object",\n        "properties": {\n          "bookingid": { "type": "integer" }\n        },\n        "required": ["bookingid"]\n      },\n      "readOnly": true,\n      "forms": [\n        {\n          "href": "https://restful-booker.herokuapp.com/booking/",\n          "op": ["readproperty"],\n          "contentType": "application/json"\n        }\n      ]\n    }\n  },\n  "actions": {},\n  "events": {}\n}'}

In [55]:
# formatting the response

import json

# Parse twice: first to extract string, second to get proper JSON
workflow_str = response['generated_td']
workflow = json.loads(workflow_str)

# Now `workflow` is a list of dicts — ready for Node-RED
print(json.dumps(workflow, indent=2))

{
  "@context": "https://www.w3.org/2019/wot/td/v1",
  "id": "urn:example:bookingService",
  "title": "BookingService",
  "security": [
    "bearer_token"
  ],
  "securityDefinitions": {
    "bearer_token": {
      "scheme": "bearer"
    }
  },
  "properties": {
    "bookings": {
      "title": "bookings",
      "description": "Returns the ids of all the bookings that exist within the API.",
      "type": "array",
      "items": {
        "type": "object",
        "properties": {
          "bookingid": {
            "type": "integer"
          }
        },
        "required": [
          "bookingid"
        ]
      },
      "readOnly": true,
      "forms": [
        {
          "href": "https://restful-booker.herokuapp.com/booking/",
          "op": [
            "readproperty"
          ],
          "contentType": "application/json"
        }
      ]
    }
  },
  "actions": {},
  "events": {}
}
